In [ ]:
def preprocess_audio(frame):
    # Convert frame to mono
    mono_frame = np.mean(frame, axis=1)
    # Extract MFCC features
    mfccs = librosa.feature.mfcc(y=mono_frame, sr=SAMPLE_RATE, n_mfcc=MFCC_COUNT)
    return np.mean(mfccs, axis=1).reshape(1, -1)


In [ ]:

def classify_sound(mfcc_features):
    prediction = model.predict(mfcc_features)
    # Assuming binary classification: 0 for non-bird, 1 for bird
    return prediction[0][0] > 0.5


In [ ]:

def audio_callback(in_data, frame_count, time_info, status):
    audio_frame = np.frombuffer(in_data, dtype=np.float32)
    mfcc_features = preprocess_audio(audio_frame)
    is_bird = classify_sound(mfcc_features)
    if is_bird:
        print("Bird sound detected!")
    return (in_data, pyaudio.paContinue)

In [ ]:


def main():
    # Initialize PyAudio
    p = pyaudio.PyAudio()
    
    # Open audio stream
    stream = p.open(format=pyaudio.paFloat32,
                    channels=1,
                    rate=SAMPLE_RATE,
                    input=True,
                    frames_per_buffer=FRAME_SIZE,
                    stream_callback=audio_callback)

    print("Recording...")
    try:
        stream.start_stream()
        while stream.is_active():
            pass
    except KeyboardInterrupt:
        print("Stopping...")
    finally:
        stream.stop_stream()
        stream.close()
        p.terminate()

if __name__ == "__main__":
    main()